In [5]:
import pandas as pd
import numpy as np
from data_preprocess import data_cleaning, data_spliting, scaling  ### Dont forget to restart the Kernel and re-import if any changes has been made to the methods ###

ds = pd.read_csv("data/antwerp_ds_weather-data_2019.csv", index_col=0)

In [6]:
x_rssi, y = data_cleaning(ds, wathear_data=False)

x_weather, y = data_cleaning(ds, wathear_data=True)

Size before cleaning:  (130429, 113)
Size after cleaning:  (55375, 113)
Size before cleaning:  (130429, 113)
Size after cleaning:  (55375, 113)


In [7]:
# make 5 different train, test and validation sets 
# the result is going to be the average of these 


In [8]:
x_rssi_scaled = scaling(scaler='MinMax', x=x_rssi)
x_w_scaled = scaling(scaler='MinMax', x=x_weather)

x_rssi_train, x_rssi_test, x_rssi_val, y_rssi_train, y_rssi_test, y_rssi_val = data_spliting(x_scaled=x_rssi_scaled, y=y, train_size=0.7, random_state=42)
x_w_train, x_w_test, x_w_val, y_w_train, y_w_test, y_w_val = data_spliting(x_scaled=x_w_scaled, y=y, train_size=0.7, random_state=42)

Training shape:      (38762, 72)
Test shape:          (8307, 72)
Validation shape:    (8306, 72)
Training shape:      (38762, 108)
Test shape:          (8307, 108)
Validation shape:    (8306, 108)


In [9]:
from sklearn.neighbors import KNeighborsRegressor

model_rssi = KNeighborsRegressor(algorithm='ball_tree')

model_rssi.fit(x_rssi_train, y_rssi_train)

y_rssi_pred = model_rssi.predict(x_rssi_test)

In [10]:
from performance_eval import *

error_stats(y_rssi_test, y_rssi_pred)

295.88544443241665

In [11]:
model_weather = KNeighborsRegressor(algorithm='ball_tree')

model_weather.fit(x_w_train, y_w_train)
y_w_pred = model_weather.predict(x_w_test)
error_stats(y_w_test, y_w_pred)

261.37364430634125

In [13]:
df1 = pd.DataFrame(y_rssi_pred, columns=['pred_lat_rssi', 'pred_lon_rssi'])
df2 = pd.DataFrame(y_w_pred, columns=['pred_lat_comb', 'pred_lon_comb'])

pd.concat([pd.DataFrame(y_rssi_test, columns=['lat', 'lon']), df1], axis=1).to_csv('files/position_pred_RSSI.csv')
pd.concat([pd.DataFrame(y_rssi_test, columns=['lat', 'lon']), df2], axis=1).to_csv('files/position_pred_weather-comb.csv')

